# Tutorial for Geocoding API and Open Street Map. 

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
# importing necessary modules
import geopandas as gpd # used for handling geospatial data
import pandas as pd # used for creating and manipulating data in tabular format 
import folium # used to create interactive maps
import requests # used to send an API call, i.e., U.S Census Geocoder
import numpy as np
from geopy.geocoders import Nominatim
from time import sleep
from datetime import datetime

## Part 1 - Converting Street Address to Lat/Lon

#### Geocoding API is a service that allows you to convert addresses into a geographic coordinates which you can use to place markers on a map. This service is currently available for the USA only. 
#### This can be achieved in batch mode i.e., multiple addresses at once and convert them to geographic coordinates. The compatible file formats to accomplish this task includes .csv, .txt, .dat, and .xlsx files. 

In [7]:
def get_coordinates(address):
    base_url = "https://geocoding.geo.census.gov/geocoder/locations/onelineaddress"
    params = {
        "address": address,
        "benchmark": "4", # Public_AR_Current
        "format": "json"
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        matches = data.get("result", {}).get("addressMatches", [])
        if matches:
            coordinates = matches[0]["coordinates"]
            return coordinates["y"], coordinates["x"]
    return None, None

### Using .txt file as an example with names of random Museums in the United States as an example. 

#### Note: this Museum Text File was randomnly generated. You Need to find a directory that has the appropriate address and Name of a destination you would like to map out. 

In [8]:
# Read the text file
file_path = 'museums.txt'
df = pd.read_csv(file_path, delimiter='\t', header=None)  # assuming "\t" tab-delimited file , replace with "," if comma seperated
# Labelling the first & second column
df.columns = ["Name", "Address"]

In [9]:
df

,Name,Address
0,The Metropolitan Museum of Art,"1000 5th Ave, New York, NY 10028"
1,The Museum of Modern Art (MoMA),"11 W 53rd St, New York, NY 10019"
2,The Art Institute of Chicago,"111 S Michigan Ave, Chicago, IL 60603"
3,Smithsonian National Museum of Natural History,"10th St. & Constitution Ave. NW, Washington, D..."
4,The Getty Center,"1200 Getty Center Dr, Los Angeles, CA 90049"
5,The National Gallery of Art,"Constitution Ave NW, Washington, DC 20565"
6,The Solomon R. Guggenheim Museum,"1071 5th Ave, New York, NY 10128"
7,The American Museum of Natural History,"Central Park West & 79th St, New York, NY 10024"
8,The Field Museum,"1400 S Lake Shore Dr, Chicago, IL 60605"
9,"The Museum of Fine Arts, Boston","465 Huntington Ave, Boston, MA 02115"


In [10]:
# Get coordinates for each address
df["Latitude"], df["Longitude"] = zip(*df["Address"].apply(get_coordinates))

# Convert to GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude), crs="EPSG:4326")

In [12]:
gdf

,Name,Address,Latitude,Longitude,geometry
0,The Metropolitan Museum of Art,"1000 5th Ave, New York, NY 10028",NaN,NaN,POINT (NaN NaN)
1,The Museum of Modern Art (MoMA),"11 W 53rd St, New York, NY 10019",40.760627,-73.976177,POINT (-73.97618 40.76063)
2,The Art Institute of Chicago,"111 S Michigan Ave, Chicago, IL 60603",41.880649,-87.624172,POINT (-87.62417 41.88065)
3,Smithsonian National Museum of Natural History,"10th St. & Constitution Ave. NW, Washington, D...",38.892079,-77.025989,POINT (-77.02599 38.89208)
4,The Getty Center,"1200 Getty Center Dr, Los Angeles, CA 90049",34.083070,-118.476108,POINT (-118.47611 34.08307)
5,The National Gallery of Art,"Constitution Ave NW, Washington, DC 20565",NaN,NaN,POINT (NaN NaN)
6,The Solomon R. Guggenheim Museum,"1071 5th Ave, New York, NY 10128",40.782857,-73.959367,POINT (-73.95937 40.78286)
7,The American Museum of Natural History,"Central Park West & 79th St, New York, NY 10024",40.775644,-73.960514,POINT (-73.96051 40.77564)
8,The Field Museum,"1400 S Lake Shore Dr, Chicago, IL 60605",41.865007,-87.614119,POINT (-87.61412 41.86501)
9,"The Museum of Fine Arts, Boston","465 Huntington Ave, Boston, MA 02115",42.339058,-71.092566,POINT (-71.09257 42.33906)


#### It is important to have the correct and specific street address. Otherwise the conversion would not work. This is one downside of geocoding. 

### Notice how a couple of Museums do not have Lat/Lon Found, this happens because the Street Address for these Museums are wrong. 

In [13]:
# Create map
m = folium.Map(location=[df["Latitude"].mean(), df["Longitude"].mean()], zoom_start=4)

# Add markers for each location
for _, row in df.iterrows():
    if not np.isnan(row["Latitude"]) and not np.isnan(row["Longitude"]):
        folium.Marker(
            location=[row["Latitude"], row["Longitude"]],
            popup=row["Address"],
            tooltip=row["Address"]
        ).add_to(m)

In [14]:
m

## Part 2 - Open Street Map Query 

### Function to automate query in Open Streeet Map using Overpass Api. 
More information about how to geocode with OSM can be found here. https://wiki.openstreetmap.org/wiki/Overpass_API?form=MG0AV3#Introduction

#### Function to pass the query to Overpass API

#### For correct Query Input visit: https://wiki.openstreetmap.org/wiki/Map_features

##### Store names like "Target", "Wallmart" wouldn't work in this case. We have to be more generic or specify Brand. 

In [15]:
def get_locations(categories, queries, cities, state, country, brand=None):
    """Fetch locations from OpenStreetMap using Overpass API."""
    overpass_url = "http://overpass-api.de/api/interpreter"
    all_locations = []

    # Construct the optional brand filter
    brand_filter = f'["brand"="{brand}"]' if brand else ''

    # Handle multiple categories, queries, and cities
    if not categories or not queries or not cities:
        print("No categories, queries, or cities provided.")
        return []

    for category in categories:
        for city in cities:
            for query in queries:
                overpass_query = f"""
                [out:json];
                area[name="{city}"]->.searchArea;
                (
                  node["{category}"="{query}"]{brand_filter}(area.searchArea);
                  way["{category}"="{query}"]{brand_filter}(area.searchArea);
                  relation["{category}"="{query}"]{brand_filter}(area.searchArea);
                );
                out center;
                """
                
                try:
                    response = requests.get(overpass_url, params={'data': overpass_query})
                    response.raise_for_status()
                    data = response.json()
                    all_locations.extend(data.get("elements", []))
                except requests.exceptions.RequestException as e:
                    print(f"Request error for {category}={query} in {city}: {e}")
                except requests.exceptions.JSONDecodeError:
                    print(f"Error decoding JSON response from API for {category}={query} in {city}.")

    return all_locations

In [16]:
def plot_locations(data, city, state, country):
    """Plot locations on a Folium map."""
    if not data:
        print("No locations found.")
        return None

    # Extract the first valid location for map centering
    for place in data:
        lat = place.get('lat') or (place.get('center', {}).get('lat'))
        lon = place.get('lon') or (place.get('center', {}).get('lon'))
        if lat and lon:
            m = folium.Map(location=[lat, lon], zoom_start=12)
            break
    else:
        print("No valid coordinates found.")
        return None

    # Add markers
    for place in data:
        lat = place.get('lat') or (place.get('center', {}).get('lat'))
        lon = place.get('lon') or (place.get('center', {}).get('lon'))
        if lat and lon:
            name = place.get('tags', {}).get('name', 'Unknown')
            folium.Marker([lat, lon], popup=f"{name} ({lat}, {lon})").add_to(m)

    return m

In [17]:
def display_locations(data):
    """Display location names with coordinates in a DataFrame."""
    locations = []
    for place in data:
        lat = place.get('lat') or (place.get('center', {}).get('lat'))
        lon = place.get('lon') or (place.get('center', {}).get('lon'))
        if lat and lon:
            name = place.get('tags', {}).get('name', 'Unknown')
            locations.append([name, lat, lon])
    
    df = pd.DataFrame(locations, columns=['Name', 'Latitude', 'Longitude'])
    return df

### Please refer to the Query Website to See which specific Keywords to use for OSM

In [18]:
# Example Query Parameters
category = ["shop"]  # General category
#queries = ["supermarket", "department_store", "greengrocer", "farm", "health_food", "retail"]  # Multiple specific queries
queries = ["supermarket"]
#cities = ["Indianapolis", "Lawrence"]  # Cities in Marion County, IN
cities = ["Indianapolis"]
state = "Indiana"
country = "USA"
brand = None  # Change to "Walmart" or "Target" if needed or None 

# Fetch Data
data = get_locations(category, queries, cities, state, country, brand)

# Plot Data on Map
map_result = plot_locations(data, cities[0] if cities else None, state, country)

# Display DataFrame of Locations
df_locations = display_locations(data)

# Display the map and data
if map_result:
    display(map_result)

display(df_locations)

,Name,Latitude,Longitude
0,Kroger,39.913798,-86.205519
1,Trader Joe's,39.912493,-86.212464
2,Kroger,39.875170,-86.119302
3,Hana Market,39.754988,-86.242418
4,Needler's Fresh Market,39.771972,-86.151670
...,...,...,...
112,El Rancho Grande,39.808881,-86.240642
113,Apna Bazaar,39.822927,-86.268925
114,Safeway,39.789167,-86.083754
115,Saraga International Grocery,39.650968,-86.120974


## Part 3 - Reverse Geo-Coding

#### (coords -> place names)

Geopy supports reverse geocoding. Handy when you have GPS points and want human-readable names.

In [22]:
# Initialize the geolocator
geolocator = Nominatim(user_agent="r_geocoder_sample")  # Replace with your app name

In [23]:
# Reverse geocode sample coordinate
reverse = geolocator.reverse((39.7684, -86.1581), exactly_one=True, timeout=10)
print("Reverse result:", reverse.address)

Reverse result: Soldiers and Sailors Monument, 1, Monument Circle, Indianapolis, Marion, Indiana, 46204, United States


## Part 4 - Batch Geocoding 

- Use a cache so repeated runs don't re-query the API.
- Respect rate limits (e.g., use RateLimiter or time.sleep).
- Consider breaking big batches into smaller chunks and run with delays.
- For production, consider a paid geocoding service with SLA and bulk endpoints.
- Save intermediate results frequently so you can resume.

In [29]:
from geopy.extra.rate_limiter import RateLimiter
import time

In [30]:
# Rate-limited geocode function: one request per second by default
geocode_rate_limited = RateLimiter(geolocator.geocode, min_delay_seconds=1, error_wait_seconds=2.0)

# Simple in-memory cache and optional disk save/load
import os, pickle
CACHE_FILE = "geocode_cache.pkl"

try:
    with open(CACHE_FILE, "rb") as f:
        geocode_cache = pickle.load(f)
    print("Loaded geocode cache with", len(geocode_cache), "entries")
except FileNotFoundError:
    geocode_cache = {}
    print("Starting with empty geocode cache")

def lookup_place(place_str, country_bias=None, limit=1, exactly_one=True):
    """
    Lookup place_str with caching. Optionally bias queries by country (ISO code or name).
    Returns geopy.Location or None.
    """
    key = (place_str, country_bias)
    if key in geocode_cache:
        return geocode_cache[key]
    try:
        query = place_str
        if country_bias:
            # Nominatim accepts 'countrycodes' param, but geopy's simple geocode may not expose all options.
            # For simple biasing we append the country name—this is heuristic.
            query = f"{place_str}, {country_bias}"
        location = geocode_rate_limited(query, exactly_one=exactly_one, timeout=10)
        geocode_cache[key] = location
        return location
    except (GeocoderTimedOut, GeocoderServiceError) as e:
        print("Geocoding error:", e)
        return None

def save_cache():
    with open(CACHE_FILE, "wb") as f:
        pickle.dump(geocode_cache, f)
    print("Cache saved:", CACHE_FILE)

Starting with empty geocode cache


In [31]:
# Example batch: input dataframe of place strings and optional context (country/state)
places_df = pd.DataFrame({
    "id": [1,2,3,4],
    "place": ["Paris", "Paris", "Springfield", "South Bend"],
    "context": [None, "TX", "IL", "IN"]  # context biases to e.g., Paris, TX
})

# We'll iterate and populate lat/lon, using cache
out_rows = []
for _, row in places_df.iterrows():
    place = row["place"]
    ctx = row["context"]
    loc = lookup_place(place, country_bias=ctx, exactly_one=False)
    if loc is None:
        out_rows.append({"id": row["id"], "place": place, "context": ctx, "found": False})
        continue
    locs = loc if isinstance(loc, list) else [loc]
    # choose top 1 for simplicity
    chosen = locs[0]
    out_rows.append({
        "id": row["id"],
        "place": place,
        "context": ctx,
        "found": True,
        "address": chosen.address,
        "lat": chosen.latitude,
        "lon": chosen.longitude
    })
    # small sleep to be polite (RateLimiter usually handles this)
    time.sleep(0.5)

batch_out = pd.DataFrame(out_rows)
print(batch_out)
#save_cache()

   id        place context  found  \
0   1        Paris    None   True   
1   2        Paris      TX   True   
2   3  Springfield      IL   True   
3   4   South Bend      IN   True   

                                             address        lat        lon  
0  Paris, Île-de-France, France métropolitaine, F...  48.858890   2.320041  
1   Paris, Lamar County, Texas, 75460, United States  33.661796 -95.555513  
2  Springfield, Sangamon County, Illinois, United...  39.799017 -89.643957  
3  South Bend, Saint Joseph County, Indiana, Unit...  41.683381 -86.250007  


## Part 5 - Handling Ambiguity

#### Handling ambiguous toponyms

A few strategies:
- **Context-based bias**: if you know the text comes from Indiana, prefer places in Indiana (country/state bias).
- **Ask for administrative level**: prefer city vs neighborhood by checking the geocoding `class` and `type` in the raw result.
- **Use bounding boxes**: Nominatim returns bounding boxes; filter results to ones whose bbox overlap a context bbox.
- **Fuzzy matching / string normalization**: normalize diacritics, punctuation, small words (e.g., "Saint" vs "St."). Use fuzzy matching to compare candidate text with geocoding address components.
- **Manual disambiguation UI**: for critical tasks, present candidate matches to a human in a small UI and ask them to pick.

In [35]:
!pip install fuzzywuzzy

In [36]:
from fuzzywuzzy import process as fuzzy_proces

In [32]:
# Example: disambiguate by simple string matching and fuzzy score
def disambiguate_candidates(candidate, n_results=5, country_bias=None):
    # Use geocode with exactly_one=False to get multiple results (the RateLimiter wrapper may return list)
    raw = lookup_place(candidate, country_bias=country_bias, exactly_one=False)
    if raw is None:
        return []
    results = raw if isinstance(raw, list) else [raw]
    choices = []
    for r in results[:n_results]:
        # create a simple score based on string similarity between candidate and the display name (address)
        score = fuzzy_proces.extractOne(candidate, [r.address or ""], score_cutoff=0)[1] if r.address else 0
        choices.append({
            "address": r.address,
            "lat": r.latitude,
            "lon": r.longitude,
            "raw": r.raw,
            "score": score
        })
    # sort by score desc
    return sorted(choices, key=lambda x: x["score"], reverse=True)

In [38]:
# Try disambiguating "Springfield" with US bias by appending "USA"
cands = disambiguate_candidates("Paris", n_results=6, country_bias="USA")
for i, c in enumerate(cands, 1):
    print(i, c["address"], c["lat"], c["lon"], "score:", c["score"])

1 Paris, Lamar County, Texas, 75460, United States 33.6617962 -95.555513 score: 60
2 Paris, Bourbon County, Kentucky, 40361, United States 38.2097987 -84.2529869 score: 60
3 Paris, Edgar County, Illinois, 61944, United States 39.611146 -87.6961374 score: 60
4 Paris, Henry County, West Tennessee, Tennessee, United States 36.3019461 -88.3258578 score: 60
5 Paris, Oxford County, Maine, 04281, United States 44.2614578 -70.5009798 score: 60
6 Paris, Logan County, Arkansas, 72855, United States 35.2924747 -93.7294452 score: 60


In [41]:
def disambiguate_candidates(place_name, context_text="", country_bias=None):
    """
    Try to disambiguate a place using context text.
    Returns a list of (location, score).
    """
    # Make a combined query
    query = f"{place_name}, {context_text}" if context_text else place_name
    print(f"🔎 Querying geocoder with: {query}")

    locs = lookup_place(query, country_bias=country_bias, exactly_one=False)

    if not locs:
        print("⚠️ No candidates found.")
        return []

    results = []
    for idx, loc in enumerate(locs):
        score = 0

        # Boost if the place_name appears in the address
        if place_name.lower() in loc.address.lower():
            score += 50

        # Boost if the context string appears in the address
        if context_text and context_text.lower() in loc.address.lower():
            score += 40

        # Small bonus for being ranked higher by Nominatim
        score += max(0, 30 - idx)

        results.append((loc, score))

    # Sort highest score first
    results.sort(key=lambda x: x[1], reverse=True)
    return results


In [54]:
# Ambiguous Springfield
candidates = disambiguate_candidates("Springfield","County", country_bias="USA")
for loc, score in candidates:
    print(score, loc.address)

# Ambiguous Paris
candidates = disambiguate_candidates("Paris", "Eiffel, Bordeaux", country_bias="FR")
for loc, score in candidates:
    print(score, loc.address)

🔎 Querying geocoder with: Springfield, County
120 Springfield, Sangamon County, Illinois, United States
119 Springfield, Hampden County, Massachusetts, United States
118 Springfield, Greene County, Missouri, United States
117 Springfield, Clark County, Ohio, United States
116 Springfield, Lane County, Oregon, United States
115 Springfield, Windsor County, Vermont, United States
114 Springfield, Washington County, Kentucky, United States
113 Springfield, Brown County, Minnesota, 56087, United States
112 Springfield, Effingham County, Georgia, United States
111 Springfield, Fairfax County, Virginia, 22150, United States
🔎 Querying geocoder with: Paris, Eiffel, Bordeaux
80 Monnaie de Paris, Avenue Gustave Eiffel, Saige, Pessac, Bordeaux, Gironde, Nouvelle-Aquitaine, France métropolitaine, 33600, France
